<a href="https://colab.research.google.com/github/chrfranco/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/7MAIR_Aprendizaje_por_Refuerzo_C_Ejercicio_de_evaluaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Teoría

- Objetivo: Repasar los conceptos vistos en clase.
- La puntuación de este bloque es de 4 puntos sobre la nota final.

Define brevemente qué es el aprendizaje por refuerzo. ¿Qué diferencias hay entre aprendizaje supervisado, no supervisado y por refuerzo?

Define con tus palabras los conceptos de Entorno, Agente, Recompensa, Estado y Observación.

Dependiendo del algoritmo de aprendizaje por refuerzo que se use, ¿qué clasificaciones podemos encontrar? Coméntalas brevemente.

Lista tres diferencias entre los algoritmos de DQN y Policy Gradient

## Práctica

- Objetivo: Implementar una solución, usando _keras-rl_ y basada en el algoritmo de _DQN_ visto en clase, para que un agente aprenda una estrategia ganadora en el juego del _Pong_.

- La puntuación de este bloque es de 6 puntos sobre la nota final.

El entorno sobre el que trabajaremos será _PongDeterministic-v0_ y el algoritmo que usaremos será _DQN_.

Para evaluar cómo lo está haciendo el agente, la recompensa en el _Pong_ oscila, aproximadamente, en el rango de valores **[-20, 20]**. La estrategia óptima de un agente estaría alrededor de una media de recompensa de 20.

- **NOTA IMPORTANTE**: Si el agente no llegara a aprender una estrategia ganadora, responder sobre la mejor estrategia obtenida.

In [0]:
# Uncomment this line for installing keras-rl on Google collaboratory
!pip install gym
 
!pip install h5py
 
!pip install Pillow
 
!pip install gym[atari]
  
!pip install jupyter
 
!pip install torch

!pip install keras-rl==0.4.2
!pip install tensorflow==1.13.1

     |████████████████████████████████| 40kB 4.4MB/s 
  Created wheel for keras-rl: filename=keras_rl-0.4.2-cp36-none-any.whl size=48379 sha256=906d6cd66de7112b8df0b0d5d0cb75c301cccff565619292db0bb48c684a53a0
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl
     |████████████████████████████████| 92.5MB 114kB/s 
     |████████████████████████████████| 368kB 50.1MB/s 
     |████████████████████████████████| 3.2MB 53.8MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
!pip list

Package                  Version    
------------------------ -----------
absl-py                  0.8.1      
alabaster                0.7.12     
albumentations           0.1.12     
altair                   3.2.0      
astor                    0.8.0      
astropy                  3.0.5      
atari-py                 0.2.6      
atomicwrites             1.3.0      
attrs                    19.3.0     
audioread                2.1.8      
autograd                 1.3        
Babel                    2.7.0      
backcall                 0.1.0      
backports.tempfile       1.0        
backports.weakref        1.0.post1  
beautifulsoup4           4.6.3      
bleach                   3.1.0      
blis                     0.2.4      
bokeh                    1.0.4      
boto                     2.49.0     
boto3                    1.10.17    
botocore                 1.13.17    
Bottleneck               1.3.0      
branca                   0.3.1      
bs4                      0.0.1      
b

In [0]:
import numpy as np
import gym
from tensorflow import keras  ## Check

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy ,BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Using TensorFlow backend.


In [0]:
ENV_NAME = 'PongDeterministic-v0'

# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
nb_actions = env.action_space.n

# random seed
np.random.seed(123)
env.seed(123)

[123, 151010689]

In [0]:
# Define the input shape to resize the screen
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 8
from PIL import Image
# This processor will be similar to the Atari processor
class PongProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        
        img = Image.fromarray(observation)
        # resize and convert to grayscale
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')  # saves storage in experience memory

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

1) Definir la arquitectura del modelo que se usará en la solución.

In [0]:
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE

model = Sequential()

model.add(Permute((2, 3, 1), input_shape=input_shape)) #Permute type tf


# TO-DO

model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

model.add(Dense(nb_actions))
model.add(Activation('linear'))

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 84, 84, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        16416     
_________________________________________________________________
activation_1 (Activation)    (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
activation_2 (Activation)    (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          36928     
________________________________________________

2) Implementación del algoritmo **_DQN_** con **_keras-rl_**.

In [0]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)# TO-DO
processor = PongProcessor() # TO-DO

In [0]:
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
                              nb_steps=1000000) # TO-DO
#policy=BoltzmannQPolicy()

In [0]:
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=20000, gamma=.99, target_model_update=1000,
               train_interval=20) # TO-DO

optimizer = Adam(lr=1e-3)# TO-DO
dqn.compile(optimizer, metrics=['mae'])

In [0]:
weights_filename = 'dqn_{}_weights.h5f'.format(ENV_NAME)
nb_steps = 50000 # TO-DO

dqn.fit(env, nb_steps=nb_steps, visualize=False)
dqn.save_weights(weights_filename, overwrite=True)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 34s 3ms/step - reward: -0.0219
10 episodes - episode_reward: -20.100 [-21.000, -17.000] - ale.lives: 0.000

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 33s 3ms/step - reward: -0.0226
11 episodes - episode_reward: -20.545 [-21.000, -19.000] - ale.lives: 0.000

Interval 3 (20000 steps performed)
   13/10000 [..............................] - ETA: 50s - reward: 0.0000e+00 WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
10000/10000 [==============================] - 130s 13ms/step - reward: -0.0235
12 episodes - episode_reward: -20.583 [-21.000, -18.000] - loss: 0.012 - mean_absolute_error: 0.059 - mean_q: -0.035 - mean_eps: 0.977 - ale.lives: 0.

In [0]:
nb_episodes = 10# TO-DO

dqn.test(env, nb_episodes=nb_episodes, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: -21.000, steps: 764
Episode 2: reward: -21.000, steps: 764
Episode 3: reward: -21.000, steps: 764
Episode 4: reward: -21.000, steps: 764
Episode 5: reward: -21.000, steps: 764
Episode 6: reward: -21.000, steps: 764
Episode 7: reward: -21.000, steps: 764
Episode 8: reward: -21.000, steps: 764
Episode 9: reward: -21.000, steps: 764
Episode 10: reward: -21.000, steps: 764


3) Justificar los resultados obtenidos y los experimentos realizados.